> **AutoGen > Old Blog (Archived) > StateFlow**
https://microsoft.github.io/autogen/0.2/blog/2024/02/29/StateFlow/


## StateFlow

**유한 상태 기계(Finite State Machine, FSM)**는 예를 들어 교통 신호 제어와 같은 실제 응용 프로그램을 모니터링하는 제어 시스템으로 사용된다. 정의된 **상태 기계**는 현재 상태를 기준으로 무엇을 해야 할지를 결정하는 행동 모델이다. *상태*는 **FSM**이 있을 수 있는 하나의 상화을 나타낸다. 이러한 개념을 바탕으로, 우리는 LLM의 작업 해결 과정을 모델링하는 데 FSM을 사용하고자 한다.

LLM을 사용하여 여러 단계를 거쳐 해결해야 하는 작업이 주어졌을 때, 작업 해결 프로세스의 각 단계를 하나의 상태로 매핑할 수 있다.
예를 들어 SQL 작업을 생각해 보자. (아래의 그림 참고) 
이 작업에 대한 바람직한 절차는 다음과 같아.

(1) 데이터베이스 내의 테이블 및 컬럼에 대한 정보 수집
(2) 원하는 정보를 조회하는 쿼리 구성
(3) 작업이 해결되었는지 검증하고, 프로세스를 종료

각 단계에 대해 대응하는 상태를 생성하고, 실패 상황을 처리하기 위한 **오류 상태(error state)**도 정의한다. 그림에서는 실행 결과에 따라 빨간 화살표(실패)와 초록 화살표(성공)가 *상태 전이*를 나타낸다. *상태간 전이*는 특정 규칙에 기반한다. 예를 들어 `"Submit"` 명령이 성공적으로 수행되면 모델은 `"종료 상태(End state)"`로 전이한다. 
상태에 도달하면, 정의된 출력함수(output functions)의 일련의 작업이 실행된다. 예를 들어 **`M_i -> E`**은 모델을 먼저 호출하고 그 이후 SQL 명령을 실행함을 의미한다.



![](https://velog.velcdn.com/images/heyggun/post/360649ab-6fcc-437a-9377-2d03df9ba54f/image.png)


위 그림은 **SQL Task**와 **Bash Task**에서의 에이전트 작업 흐름을 시각적으로 표현한 다이어그램이다. 각 작업에서 **LLM(Large Language Model)이 특정 instruciton을 가진 에이전트로 동작하면서 오류를 탐지하고 수정하면서 해결하는 과정이다.
 
각 기호들은 다음과 같다.
- **Init** : 초기 프롬프트 입력 (P->E, *Prompter* -> *Execution*)
- **M_i** -> E : *M_i*는 특정 instrauction을 따르는 LLM, *E*는 실행환경이다. 
- **초록색 원** : 작업 수행 단계 (정상 플로우)
- **빨간색 원** : 오류 처리 단계 (Error)
- **화살표** : 단계 간 전이
- **Exit** : 작업 제출 및 종료 


**[SQL Task]**

SQL Task의 전체 흐름은 *Init* 단계에서 시작해서 *Observe(M_1->E)*로 데이터를 관찰한다. *Select (M_2->E)* 로 쿼리를 선택하고 *Verify(M_3->E)* 선택한 쿼리를 검증한다. 그 후에 *End*로 제출하면 성공적인 종료이다.

이 때, 중간에 오류가 발생하면 오류 처리 플로우로 빠지는데 *Obseve*,*Select*, *Verify* 단계에서 **Error(M_4->E)**로 전환될 수 있다. Error는 자체 루프를 돌며 재시도가 가능하고, Error에서 *Select*나 *Verify*로 복귀할 수 있다.

해당 로직에서 *Select* <-> *Verify* 에서 **Select**, **Desc** 키워드를 기반으로 상호 피드백을 진행한다.

**[Bash Task]**

Bash Task의 전체흐름은 *Init*에서 시작해서 *Solve(M_1->E)* 에서 문제 해결을 시도한다. 그리고 *Verify(M_2->E)*로 실행 결과를 확인한 후에 *End*로 제출하면 성공적인 종료이다.

이 때, 중간에 오류가 발생하면 *Solve*, *Verify* 단계에서 **Error(M_3->E)**로 전환한다. Error는 역시 루프로 반복 가능하다. 오류 발생 시 다시 *Verity*로 돌아가거나, 처음부터 다시 *Sovle*로 돌아갈 수 있다. 

위의 로직들의 핵심 차이점은 **SQL task**는 주요단계가 `Observe->Select->Verify` 이고, **Bash Task**는 주요단계가 `Solve -> Verify` 이다. **SQL task**는 검증-선택의 반복 단계에서 `SELECT/DESC`로 상호 피드백을 진행하고 **Bash Task**는 단방향 흐름 중심이다. 

이 다이어그램은 **에이전트 기반 문제 해결** 프로세스에서 `다단계 피드백 루프`, `오류 처리 로직`, `역할 분리된 모델` 사용 전략을 보여주고 있다. 특히 **SQL 문제 해결**은 **탐색-선택-검증**의 명확한 단계화를 통해 구조적 문제를 해결을 강조하고, Bash는 **실행 중심**의 간단한 흐름으로 실용적인 접근을 보여준다. 

## Experiments


**InterCode** : 여기서는 InterCode 벤치마크의 SQL 작업과 Bash 작업에서 StateFlow를 평가하며, GPT-3.5-Turbo와 GPT-4-Turbo 모두를 사용했다. 포괄적인 비교를 위해 다양한 지표를 기록했다.
**`'SR(성공률)'`**은 성능을 측정하고, **`'Truns(턴수)'`**는 환경과의 상호작용 횟수를 나타내며, **`'Error Rate(오류율)'`**는 실행된 명령어 중 오류가 발생한 비율을 나타낸다. 또한 LLM 사용 비용도 기록하였다.
다음과 같은 기준 방법으로 비교를 진행하였는데, 

(1) **ReAct** : 모델이 생각과 행동을 생성하도록 유도하는 few-shot 프롬프트 방식
(2) **Plan & Solve** : 먼저 모델에게 계획을 제안하게 한 다음, 이를 실행하게 하는 두 단계 프롬프트 전략

Bash 작업에 대한 결과는 아래에 제시되어 있다.
![](https://velog.velcdn.com/images/heyggun/post/0cd11ea0-f023-4bab-b1ac-60d8920db4de/image.png)


**ALFWorld** : 여기서는 **TextWorld** 환경에서 구현된 텍스트 기반의 합성 게임인 `ALFWorld 벤치마크`에서도 실험을 수행했다. `GPT-3.5-Turbo`를 사용했으며, 평균 3회의 시도에서 결과를 측정했다. 다음의 방법들로 함께 평가를 진행했다.

(1) **ReAct** : ReAct의 two-shot 프롬프트를 사용하였으며, 각 작업 유형에 따라 별도의 프롬프트를 사용한다.
(2) **ALFChat (2 에이전트)**: AutoGen의 설정을 따른 두 개의 에이전트 시스템으로, 보조 에이전트와 실행 에이전트로 구성된다. ALFChat은 ReAct를 기반으로 하며, 프롬프트를 대화 형식으로 수정한 방식이다.
(3) **ALFChat (3 에이전트)** : 2-에이전트 시스템을 기반으로 하여, 보조 에이전트가 동일한 행동을 세 번 연속 출력할 경우 상식 정보를 제공하는 grounding 에이전트를 추가한 방식이다.

두 작업 모두에서 **StateFlow**는 가장 높은 성능과 가장 낮은 비용을 달성했다. 자세한 내용은 본 논문을 참고 !

> **paper : StateFlow: Enhancing LLM Task-Solving through State-Driven Workflows**
https://arxiv.org/abs/2403.11322

## Implement StateFlow With GroupChat

여기서는 *GroupChat*을 사용해 **StateFlow**를 구축하는 방법을 설명한다. **`FSM Group Chat`**에서는 에이전트 간 전환을 제약할 수 있는 `transition graph)` 전이 그래프를 입력하는 새로운 GroupChat 기능을 소개했다. 이 기능은 에이전트의 설명(description) 파마미터에 상태 전이 조건을 자연어로 기술 해야 하며, LLM이 해당 설명을 받아 다음 에이전트를 결정하게 된다.

> **FSM Group Chat**
https://microsoft.github.io/autogen/0.2/blog/2024/02/11/FSM-GroupChat/


여기서는 **GroupChat** 객체의 `speaker_selection_method`에 전달할 수 있는 **사용자 지정 발화자 선택 함수(custom speaker selection function)**을 사용한다. 이 함수는 에이전트 간의 전이 로직을 사용자 정의할 수 있게 해주며, FSM Group Chat에서 소개된 전이 그래프와 함께 사용할 수 있다. 현재 StateFlow 구현은 사용자가 전이 그래프를 **직접 덮어쓸 수 있도록**도 지원한다. 이러한 전이는 현재 발화자(speaker)와 컨텍스트 히스토리의 정적 검사(예: 마지막 메시지에 'Error'가 포함되어 있는지 확인)를 기반으로 정의될 수 있다. 

**GroupChat**을 사용해서 **`state-oriented workflow(상태 지향 워크플로우)`**를 구성하는 예제를 제시한다. 여기서는 `speaker_selection_method` 파라미터에 전달될 사용자 정의 발화 선택 함수를 정의한다. 이 예제의 작업은 주어진 주제와 관련된 연구 논문을 검색하고, 해당 논문들을 마크다운 테이블 형식으로 정리하는 것이다.

![](https://velog.velcdn.com/images/heyggun/post/b9145a1e-3ae6-4661-a9b7-007798d1d1b7/image.png)

위는 코드를 생성하고 실행해서 추가 연구를 통해 작업을 완성하는 일련의 프로세스를 설명하는 다이어그램이다. 

*Init* 프로세스를 시작하는 초기화 단계에서 시작해서 *Retrieve(**C->E**)* 코드를 작성(Coder)하고 코드를 실행한다(Coe Executor). *Research(**R**) *성공적으로 결과를 얻으면 추가적인 리서치를 진행한다. 연구가 끝나면 *End*(**Exit**) 작업을 종료한다.
위 로직에서 Coder가 코드를 작성하고 Code Executor라 이를 실행해서 실행에 성공하면 *Research* 단계로, 오류 발생시 *Retrieve* 단계로 다시 돌아가서 재 시도한다. (Error 루프)

---
여기서는 다음과 같은 에이전트를 정의했다.

- **Initializer** : 작업을 전달하여 워크플로우를 시작한다.
- **Coder**: 코드를 작성하여 인터넷에서 논문을 검색한다.
- **Executor** : 코드를 실행한다.
- **Scientist** : 논문을 읽고 요약을 작성한다.


In [3]:
import autogen
from config import settings

In [4]:
api_key = settings.openai_api_key.get_secret_value()

In [5]:
llm_config = {
    "config_list":
        [
            {
                "model" : "gpt-4o-mini",
                "api_key" : api_key
            }
        ]}

In [6]:
initializer = autogen.UserProxyAgent(
    name="init",
    llm_config=llm_config,
)

coder = autogen.AssistantAgent(
    name="Coder",
    system_message="""You are the Coder. Write Python Code to retrieve papers from arxiv.""",
    llm_config=llm_config,
)

executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="""Executor. Execute the code written by the Coder and report the result.""",
    llm_config=llm_config,
)

scientist = autogen.AssistantAgent(
    name="Scientist",
    system_message = """You are the Scientist. Please categorize papers after seeing their abstracts printed and create a markdown table with Domain, Title, Authors, Summary and Link.
Return 'TERMINATE' in the end.""",
llm_config=llm_config,)

- **Initializer** : 작업을 전달하여 워크플로우를 시작한다.
- **Coder**: 코드를 작성하여 인터넷에서 논문을 검색한다.
- **Executor** : 코드를 실행한다.
- **Scientist** : 논문을 읽고 요약을 작성한다.


In [7]:
def state_transition(last_speaker, groupchat):
    messages = groupchat.messages
    
    if last_speaker is initializer:
        return coder
    elif last_speaker is coder:
        return executor
    elif last_speaker is executor:
        if messages[-1]["content"] == "exitcode: 1":
            return coder
        else:
            return scientist
    elif last_speaker == "Scientist":
        return None
    

In [8]:
groupchat = autogen.GroupChat(
    agents=[initializer, coder, executor, scientist],
    messages=[],
    max_round=20,
    speaker_selection_method=state_transition,
)


In [9]:
manager = autogen.GroupChatManager(groupchat=groupchat)

initializer.initiate_chat(manager)

init (to chat_manager):



--------------------------------------------------------------------------------

Next speaker: Coder

Coder (to chat_manager):

To retrieve papers from arXiv, you can use the `arxiv` API, which allows you to search for and fetch papers in a variety of formats. Below is a simple example of how to use the `arxiv` Python package to retrieve papers. Make sure to install the package first if you haven't done so:

```bash
pip install arxiv
```

Here's a sample code snippet to search for papers on arXiv:

```python
import arxiv

def search_arxiv(query, max_results=5):
    # Create a search object with the specified query and max results
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance
    )
    
    # Iterate through the search results and print paper details
    for result in search.results():
        print(f'Title: {result.title}')
        print(f'Authors: {", ".join([author.name for au

ChatResult(chat_id=None, chat_history=[{'content': '', 'role': 'assistant', 'name': 'init'}, {'content': 'To retrieve papers from arXiv, you can use the `arxiv` API, which allows you to search for and fetch papers in a variety of formats. Below is a simple example of how to use the `arxiv` Python package to retrieve papers. Make sure to install the package first if you haven\'t done so:\n\n```bash\npip install arxiv\n```\n\nHere\'s a sample code snippet to search for papers on arXiv:\n\n```python\nimport arxiv\n\ndef search_arxiv(query, max_results=5):\n    # Create a search object with the specified query and max results\n    search = arxiv.Search(\n        query=query,\n        max_results=max_results,\n        sort_by=arxiv.SortCriterion.Relevance\n    )\n    \n    # Iterate through the search results and print paper details\n    for result in search.results():\n        print(f\'Title: {result.title}\')\n        print(f\'Authors: {", ".join([author.name for author in result.authors]

In [10]:
for msg in manager.groupchat.messages:
    print(f"{msg['role']}: {msg['content']}")

user: 
user: To retrieve papers from arXiv, you can use the `arxiv` API, which allows you to search for and fetch papers in a variety of formats. Below is a simple example of how to use the `arxiv` Python package to retrieve papers. Make sure to install the package first if you haven't done so:

```bash
pip install arxiv
```

Here's a sample code snippet to search for papers on arXiv:

```python
import arxiv

def search_arxiv(query, max_results=5):
    # Create a search object with the specified query and max results
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance
    )
    
    # Iterate through the search results and print paper details
    for result in search.results():
        print(f'Title: {result.title}')
        print(f'Authors: {", ".join([author.name for author in result.authors])}')
        print(f'Summary: {result.summary}')
        print(f'Published: {result.published}')
        print(f'PDF Lin